In [1]:
!nvidia-smi

Tue Feb 20 13:44:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.52                 Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   48C    P8              4W /   50W |     230MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


## working on wizards of oz for bigram model

In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


#### setting up basic encoding and decoding 

In [4]:

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

#### plitting the data

In [5]:
print(block_size, "\n",batch_size)

8 
 4


In [6]:
n=int(0.8*len(data))
train_data=data[:n]
test_data=data[n:]


In [7]:
len(data)-block_size

232301

In [8]:
data=train_data
ix=torch.randint(len(data)-block_size, (batch_size,))
print(ix)
x=torch.stack([data[i:i+block_size] for i in ix])
y=torch.stack([data[i+1:i+block_size+1] for i in ix])
print(x)
print("\n",y)

tensor([ 33555,  94048, 110785,  86004])
tensor([[72,  1, 67, 68, 71,  1, 69, 58],
        [ 1, 76, 54, 72,  1, 54,  1, 60],
        [67, 78, 73, 61, 62, 67, 60,  1],
        [68, 73, 61, 78, 11,  0,  0, 44]])

 tensor([[ 1, 67, 68, 71,  1, 69, 58, 68],
        [76, 54, 72,  1, 54,  1, 60, 62],
        [78, 73, 61, 62, 67, 60,  1, 65],
        [73, 61, 78, 11,  0,  0, 44, 61]])


In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

tensor([ 15476, 105254, 111273,  93619])
inputs:
tensor([[ 1, 61, 58, 54, 57,  1, 68, 75],
        [73,  1, 72, 61, 58,  1, 54, 73],
        [ 3,  0,  0, 44, 61, 58, 78,  1],
        [67, 68, 73,  2,  3,  1, 54, 57]], device='cuda:0')
targets:
tensor([[61, 58, 54, 57,  1, 68, 75, 58],
        [ 1, 72, 61, 58,  1, 54, 73, 58],
        [ 0,  0, 44, 61, 58, 78,  1, 73],
        [68, 73,  2,  3,  1, 54, 57, 57]], device='cuda:0')


### training the bigram model

### >> Understan the view--> it is like unpack

In [15]:
a=torch.rand(2,3,4)
print(a.shape)
x,y,z=a.shape
k=a.view(x,y,z)
print(k.shape)

torch.Size([2, 3, 4])
torch.Size([2, 3, 4])


In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            print("Logits shape is :",B, T, C)
            logits = logits.view(B*T, C) ## kind of reshape as pytorch expects it in (N,C) way: N== Batch and C == classes
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
                                   ##torch.long--> int 64 
context = torch.zeros((1,1), dtype=torch.long , device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


&!rDLDcSsm7"Cpq:CiJ)g0VOvJ&!n4
SF:HlpL5qSeE)t]?7Q7&V6sen]XudE"91"4V7q2udC[:?sWvj SjQqEFsCq3:gYT7D5(wR3N﻿Y1﻿:.whtT!i(38F)G"P3c-.XAZKD*w3a]Qq﻿Q:.C*NoBUZeoU'f4":'
fj
&SnWKNeJwoNuRTTM2LE-EK﻿4jgXm(OdbN﻿gAVso?3﻿0TjIG?h36kr5hp)7*dZKgOX?"?h,l:rOjO)LJXLwGP_
Yt3pLT7an﻿gq2Xy8﻿m)G"
wo2f-BrgFqQATOHxB FqIbDG"*24eThcr!0Sf]!omAOj,dG]]OB8l&oRyVjGNB6fly]ilgtm(c7'Akxz'&(&M(Ua])tJE!_:Ejp,5ITNpN0E*'DoUyTMyTfJa0,VTqurWB5SoChc,"fxbqB5kxb1YtaSfy-b(;iKstvRa5qUZXqv*IU;XXueN!nSzqvq[rmVl84ax1SbR!ZG]?J(c(psdjXm8ET7*2!,:tttg


## This above geenration is just random and is wihtout training 
>> As we can see, it doesn't make sense
</br> *Lets use optimizer to train the model*

In [20]:
# model.forward.logits

In [12]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

In [22]:
## decalring learning rate
l_r=3e-4
max_iters=10000

#### creating a Pytorch Optimizer

In [23]:
optmizer=torch.optim.AdamW(model.parameters(),lr=l_r)

for inter in range(max_iters):
    xb,yb=get_batch("train")
    logits,loss=model.forward(xb,yb)
    optmizer.zero_grad(set_to_none=True)
    loss.backward()
    optmizer.step()
print(loss.item())

tensor([ 95105,  62151,  82941, 141755])
Logits shape is : 4 8 81
tensor([131250, 134045, 104709,  96166])
Logits shape is : 4 8 81
tensor([112843,  79148, 141364,  65526])
Logits shape is : 4 8 81
tensor([73822, 77064, 71353, 61321])
Logits shape is : 4 8 81
tensor([113174,  69353, 135483,  39760])
Logits shape is : 4 8 81
tensor([ 10617,  14560, 100697,  94976])
Logits shape is : 4 8 81
tensor([136903, 132402,  82431,  19468])
Logits shape is : 4 8 81
tensor([119240,  23945,  23612,  49036])
Logits shape is : 4 8 81
tensor([102569,  19407,  90713,  79945])
Logits shape is : 4 8 81
tensor([ 75575, 112642,  87625, 124563])
Logits shape is : 4 8 81
tensor([ 91526,  95628, 143000, 117752])
Logits shape is : 4 8 81
tensor([ 78213,  51318, 124180,  44589])
Logits shape is : 4 8 81
tensor([ 56872,  82304, 108298,  19619])
Logits shape is : 4 8 81
tensor([132076,  30039,  64356,  22113])
Logits shape is : 4 8 81
tensor([ 77076,  24892, 122360, 123840])
Logits shape is : 4 8 81
tensor([138855

#### it will still be grabage as it is just a bigram model and wont generate effective "tokens",</br> so we are just testing things out

In [24]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


HgI h
c7&.
an]3NQzSAT8fe EWHSvG:(xkploAoWG"Y6kyus[osaLSb &5WX
a
u3
0s the'5.80py-H"Wcan:bysqUVD.gy gruf!N_pep,lell I;Uhmgr ex,Gd iPKvXw3FUDEor.O0,& PJOvY*x,,:(ur hr ecqdingn)9]WBcs led ]trg6h
WRouseyTA
ms,iaruensl Ih ulkzMutoox﻿;t,lsoais
8JgP&&urk!11gjL_j.(nwvU[ov* flsh EG?yP9,&urHiTNXUJ]xn tax,YhO5VCEomofiATO
hek't ;bed
9Rwfth,h2;senySk Hu, ou'2WB!Y;zntsefCX;.-_T4
aQ2q:jGQ,l,Sf..5A8bfgr7V8t gUJk)Verug2uVvY6hr y JWBAncky8o xzt i;UYhy 7VFLCP6Ov[C038)_:6X;OBFq2;ankAYthela03"
Jy
vduP6
"bey1&Nl 1.
9


#### Let's now, check how our model  is converging over some time of iterations

In [25]:
eval_iteration=250
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [27]:

optmizer=torch.optim.AdamW(model.parameters(),lr=l_r)

for inter in range(max_iters):
    if inter % eval_iteration == 0:
        losses = estimate_loss()
        print(f"step: {inter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    xb,yb=get_batch("train")
    logits,loss=model.forward(xb,yb)
    optmizer.zero_grad(set_to_none=True)
    loss.backward()
    optmizer.step()
print(loss.item())


tensor([83923, 87822, 72934, 32072])
Logits shape is : 4 8 81
tensor([129117, 147312,  34382,  56093])
Logits shape is : 4 8 81
tensor([142857, 111355,  63856, 139307])
Logits shape is : 4 8 81
tensor([ 98859, 112074,  97015,  74923])
Logits shape is : 4 8 81
tensor([ 10437, 130858, 129324,  17996])
Logits shape is : 4 8 81
tensor([96263, 16391, 63549, 46091])
Logits shape is : 4 8 81
tensor([114837,  40506,  80505,  22773])
Logits shape is : 4 8 81
tensor([97063, 99119, 38846, 19374])
Logits shape is : 4 8 81
tensor([ 11398,  10573,  50451, 117682])
Logits shape is : 4 8 81
tensor([104294,  79705,  88638,  47189])
Logits shape is : 4 8 81
tensor([  5451,  46411, 144264, 141424])
Logits shape is : 4 8 81
tensor([ 83205,  48437, 113687,  50862])
Logits shape is : 4 8 81
tensor([135502,   8508,  92396,  66487])
Logits shape is : 4 8 81
tensor([90306, 38443, 50223, 66546])
Logits shape is : 4 8 81
tensor([ 96870, 104885, 124506,  97940])
Logits shape is : 4 8 81
tensor([ 41142, 118972, 14